<a href="https://colab.research.google.com/github/michellecxy015/search-engines/blob/main/code/google_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas torch torchvision transformers Pillow

In [ ]:
import pandas as pd
import base64
from io import BytesIO
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# Load CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Define categories with prompts
categories = {
    "safe for children": [
        "This image is suitable for children.",
        "This image contains friendly and educational content.",
        "This image is child-appropriate.",
        "This image contains no harmful or inappropriate content."
    ],
    "violence": [
        "This image depicts violence.",
        "This image contains weapons.",
        "This image depicts fights or bullying.",
        "This image contains blood or gore."
    ],
    "sexual content": [
        "This image contains nudity.",
        "This image is sexually explicit.",
        "This image is pornographic.",
        "This image contains inappropriate sexual content."
    ],
    "scary or shocking": [
        "This image is scary or disturbing.",
        "This image depicts horror or fear.",
        "This image is dark and creepy.",
        "This image is shocking or disgusting."
    ]
}

# Function to process image from "Image URL" (Base64 or HTTP URL)
def process_image(src):
    if src.startswith("data:image"):  # Base64-encoded image
        base64_data = src.split(",")[1]  # Extract Base64 data
        image_data = base64.b64decode(base64_data)
        image = Image.open(BytesIO(image_data)).convert("RGB")
    elif src.startswith("http"):  # URL of the image
        response = requests.get(src, stream=True)
        if response.status_code == 200:
            image = Image.open(response.raw).convert("RGB")
        else:
            raise ValueError(f"Failed to fetch image from URL: {src}")
    else:
        raise ValueError("Unsupported image format.")
    return image


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:

# Function to classify an image using CLIP
def classify_image(image):
    # Flatten prompts
    prompts = []
    labels = []
    for category, descriptions in categories.items():
        prompts.extend(descriptions)
        labels.extend([category] * len(descriptions))

    # Process inputs for CLIP
    inputs = processor(text=prompts, images=image, return_tensors="pt", padding=True)
    outputs = model(**inputs)

    # Get logits and probabilities
    logits_per_image = outputs.logits_per_image  # Logits for image-text pairs
    probs = logits_per_image.softmax(dim=1)  # Convert to probabilities

    # Calculate summed probabilities
    category_probs_sum = {}
    for i, label in enumerate(labels):
        category_probs_sum[label] = category_probs_sum.get(label, 0) + probs[0][i].item()

    # Find the category with the maximum summed probability
    most_likely_label = max(category_probs_sum, key=category_probs_sum.get)
    confidence = category_probs_sum[most_likely_label]

    return most_likely_label, confidence, category_probs_sum

In [ ]:
csv_path = "/content/drive/MyDrive/filter_image_metadata.csv"  # Replace with your actual CSV file path
data = pd.read_csv(csv_path)
# Specify the columns to check for 0 values
columns_to_check = ["X", "Y", "Width", "Height"]

# Filter out rows where any of the specified columns have 0 values
filtered_data = data[(data[columns_to_check] != 0).all(axis=1)]

# Print the number of rows removed
rows_removed = data.shape[0] - filtered_data.shape[0]
print(f"Number of rows removed: {rows_removed}")

# Save the filtered dataset to a new CSV file
output_csv_path = "/content/drive/MyDrive/filtered_data_cleaned.csv"
filtered_data.to_csv(output_csv_path, index=False)
print(f"Filtered data saved to {output_csv_path}.")


Number of rows removed: 185
Filtered data saved to /content/drive/MyDrive/filtered_data_cleaned.csv.


In [ ]:

# Load the CSV file
csv_path = "/content/drive/MyDrive/filtered_data_cleaned.csv"  # Replace with your actual CSV file path
data = pd.read_csv(csv_path)

# Initialize results lists
labels = []
confidences = []
categories_probs = []
safe_60_flags = []
safe_70_flags = []
safe_80_flags = []
safe_50_flags = []

# Classify each image
for i, row in data.iterrows():
    try:
        # Process the image and classify
        image = process_image(row["Image URL"])  # Replace "Image URL" with your column name
        label, confidence, category_probs = classify_image(image)

        # Append classification results
        labels.append(label)
        confidences.append(confidence)
        categories_probs.append(category_probs)

        # Apply thresholds
        safe_60_flags.append(label == "safe for children" and confidence >= 0.6)
        safe_70_flags.append(label == "safe for children" and confidence >= 0.7)
        safe_80_flags.append(label == "safe for children" and confidence >= 0.8)
        safe_50_flags.append(label == "safe for children" and confidence >= 0.5)

    except Exception as e:
        print(f"Error processing row {i}: {e}")
        labels.append("Error")
        confidences.append(0)
        categories_probs.append({})
        safe_60_flags.append(False)
        safe_70_flags.append(False)
        safe_80_flags.append(False)
        safe_50_flags.append(False)

# Add results to the DataFrame
data["most_likely_label"] = labels
data["confidence"] = confidences
data["category_probs"] = categories_probs
data["safe_60"] = safe_60_flags
data["safe_70"] = safe_70_flags
data["safe_80"] = safe_80_flags
data["safe_50"] = safe_50_flags

# Save the updated DataFrame to a new CSV file
output_csv_path = "/content/drive/MyDrive/filtered_classified_image_data_label.csv"
data.to_csv(output_csv_path, index=False)

In [ ]:
csv_path = "/content/drive/MyDrive/unfilter_image_metadata.csv"# Replace with your actual CSV file path
data = pd.read_csv(csv_path)
# Specify the columns to check for 0 values
columns_to_check = ["X", "Y", "Width", "Height"]

# Filter out rows where any of the specified columns have 0 values
filtered_data = data[(data[columns_to_check] != 0).all(axis=1)]

# Print the number of rows removed
rows_removed = data.shape[0] - filtered_data.shape[0]
print(f"Number of rows removed: {rows_removed}")

# Save the filtered dataset to a new CSV file
output_csv_path = "/content/drive/MyDrive/unfiltered_data_cleaned.csv"
filtered_data.to_csv(output_csv_path, index=False)
print(f"Filtered data saved to {output_csv_path}.")

Number of rows removed: 145
Filtered data saved to /content/drive/MyDrive/unfiltered_data_cleaned.csv.


In [ ]:

# Load the CSV file
csv_path = "/content/drive/MyDrive/unfiltered_data_cleaned.csv"  # Replace with your actual CSV file path
data = pd.read_csv(csv_path)

# Initialize results lists
labels = []
confidences = []
categories_probs = []
safe_60_flags = []
safe_70_flags = []
safe_80_flags = []
safe_50_flags = []

# Classify each image
for i, row in data.iterrows():
    try:
        # Process the image and classify
        image = process_image(row["Image URL"])  # Replace "Image URL" with your column name
        label, confidence, category_probs = classify_image(image)

        # Append classification results
        labels.append(label)
        confidences.append(confidence)
        categories_probs.append(category_probs)

        # Apply thresholds
        safe_60_flags.append(label == "safe for children" and confidence >= 0.6)
        safe_70_flags.append(label == "safe for children" and confidence >= 0.7)
        safe_80_flags.append(label == "safe for children" and confidence >= 0.8)
        safe_50_flags.append(label == "safe for children" and confidence >= 0.5)

    except Exception as e:
        print(f"Error processing row {i}: {e}")
        labels.append("Error")
        confidences.append(0)
        categories_probs.append({})
        safe_60_flags.append(False)
        safe_70_flags.append(False)
        safe_80_flags.append(False)
        safe_50_flags.append(False)

# Add results to the DataFrame
data["most_likely_label"] = labels
data["confidence"] = confidences
data["category_probs"] = categories_probs
data["safe_60"] = safe_60_flags
data["safe_70"] = safe_70_flags
data["safe_80"] = safe_80_flags
data["safe_50"] = safe_50_flags

# Save the updated DataFrame to a new CSV file
output_csv_path = "/content/drive/MyDrive/unfiltered_classified_image_data_with_label.csv"
data.to_csv(output_csv_path, index=False)
